Task1: Network Identification
Consider the network N represented in the file net_x, where x is your group number.
This network has been generated with one of the network models seen during the course.
You have to analyze the network N with the network mining tools (the ones shortlisted in the
midterm project) and guess which model has been used for creating it. Your guess has to be
supported by an appropriate set of experiments to confirm that networks generated with the
proposed model have characteristics similar to N (note that you have to guess also the parameters of
the model).
During the discussion of the project, you will be asked to motivate your guess. Motivations may be
related to both theoretical properties of the models seen during the course (e.g., “I analyzed the
provided network and I observed that its node degree distribution follows a power law. Hence, I
conclude that it is not possible that the graph has been generated with a model random(n, p).”), and
to experimental evidence (e.g., “I generated a lot of random graphs with p = 1/3, and none of them
had similar properties as the provided network. Hence I conclude that it is improbable that the graph
is random(n, 1/3)”).
A bonus point will be assigned to all the components of the groups whose guess is closer to the
model (and parameters) used to generate N.


In [21]:
# Task 1:
import networkx as nx
from joblib import Parallel, delayed, cpu_count
import pandas as pd
import itertools as it
import math
from scipy.sparse import linalg
import random
from sklearn.cluster import SpectralClustering
import numpy as np


Different types of graphs it can be:
configurationG(deg)
power_law_degree(n,power)
preferentialG(n,p)
GenWS2DG(n, r, k, q)
affiliationG(n, m, q, c, p, s)
randomG(n,p)


info:
Is undirected
follows a normal distribution of neighbors with mean 23.3

Parameters:
Nodes: 20000
edges: 227739


In [22]:
G = nx.Graph()

with open('net_3', 'r') as f:
    for s in f.readlines():
        G.add_edge(s.split()[0], s.split()[1])


print(G.number_of_nodes())
print(G.number_of_edges())

# graph is connected
print(nx.is_connected(G))

# Average degree of neighbors in the graph, tells us that on average a neihgbor of a node has 23.24 connections
# which is a very connected graph
mean_deg = print(np.mean(list(nx.average_neighbor_degree(G).values())))

#print(nx.node_connectivity(g))

s = nx.degree_alg.degree_centrality(G)
max_k = '0'
min_k = '0'
for k in s:
    if s[k] >= s[max_k]:
        max_k = k
    if s[k] <= s[min_k]:
        min_k = k   
print(max_k, s[max_k]) # very low centrality overall, max is 0.0021
print(min_k, s[min_k]) # min is 0.00020, which is 10 times lowe than the max

#print(nx.diameter(g)) # Should take some time to compute. Value is:
a = []
for i in G.nodes:
    a.append(len(G[i]))

std_deg = np.std(a)



20000
227739
True
23.24426676942118
1424 0.0021001050052502626
10443 0.000200010000500025


In [23]:
# function to get the different kinds of centralities
import networkx as nx
def centrality(G):
    # degree centrality
    d_cen = nx.degree_centrality(G) # takes 5 seconds
    print('degree done')
    # closeness centrality
    c_cen = nx.closeness_centrality(G) # takes 12 minutes
    print('closeness done')
    # betweenness centrality
    b_cen = nx.betweenness_centrality(G) # takes forever 50
    print('betweenness done')
    # eulerian centrality
    #e_cen = nx.eigenvector_centrality(G)
    #print('eulerian done')
    # return the different centralities
    return [[min(d_cen),max(d_cen)], [min(c_cen),max(c_cen)], [min(b_cen),max(b_cen)]]

In [24]:
df = pd.DataFrame()
df['nodes'] = G.nodes
df['number of neighbors'] = [len(G[i]) for i in df['nodes']]
#df['centrality'] = [nx.centrality.closeness.closeness_centrality(g,i) for i in df['nodes']]
print(np.std(df['number of neighbors']))
print(np.mean(df['number of neighbors']))

5.148997843269395
22.7739


In [25]:
dfG = pd.DataFrame(centrality(G), columns=['Degree', 'Closeness', 'Betweenness'])

degree done
closeness done


In [ ]:
import seaborn as sns

hist_facit = sns.histplot(df['number of neighbors'])
sns.PairGrid.savefig(hist_facit,'./figures/histo_facit.png')


In [ ]:

mean_deg = 23
num_nodes = 20000
std_dev = 5
degrees = np.round(np.random.normal(mean_deg, std_dev, num_nodes)).astype(int)

if sum(degrees) % 2 != 0:
    degrees[0]-=1
g = nx.configuration_model(degrees)


In [ ]:
dfg = pd.DataFrame(centrality(g))

In [ ]:
s = nx.degree_alg.degree_centrality(G)
max_k = '0'
min_k = '0'
for k in s:
    if s[k] >= s[max_k]:
        max_k = k
    if s[k] <= s[min_k]:
        min_k = k   
print('Centrality for net_3')
print(max_k, s[max_k]) # very low centrality overall, max is 0.0021
print(min_k, s[min_k]) # min is 0.00020, which is 10 times lowe than the
s = nx.degree_alg.degree_centrality(g)
max_k = 0
min_k = 0
for k in s:

    if s[k] >= s[max_k]:
        max_k = k
    if s[k] <= s[min_k]:
        min_k = k   

print('Centrality for approximation')
print(max_k, s[max_k]) # very low centrality overall, max is 0.0021
print(min_k, s[min_k]) # min is 0.00020, which is 10 times lowe than the

print('Same centrality')

In [ ]:
import seaborn as sns
df = pd.DataFrame()
df['nodes'] = g.nodes
df['Number of neighbors'] = [len(g[i]) for i in df['nodes']]
histo_fit = sns.histplot(df['Number of neighbors'])
histo_fit.set_title(f'Mean = {mean_deg}, Std = {std_dev}')
sns.PairGrid.savefig(histo_fit,'./figures/histo_fit.png')



In [ ]:
dfg

,degree,closeness,betweenness,eigenvector,page
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
